In [4]:
import pandas as pd

df = pd.read_json('crawler/output.json')

In [5]:
df.head()

,url,title,content
0,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"NvidiaSearchInput.mount({""apiUrl"": ""https://ap..."
1,https://docs.nvidia.com/cuda/cuda-toolkit-rele...,CUDA 12.5 Update 1 Release Notes,1. CUDA 12.5 Update 1 Release Notes \r\n 1.1. ...
2,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Linux,1. Introduction \n 1.1. System Requirements \n...
3,https://docs.nvidia.com/cuda/cuda-features-arc...,CUDA Features Archive,1. CUDA 11.6 Features \n \n 1.1. Compiler \n 1...
4,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Microsoft Windows,1. Introduction \n 1.1. System Requirements \n...


In [6]:
df["complete_texts"] = df.apply(lambda x: str(x["title"])+"\n"+str(x["content"]), axis=1)

df.head()

,url,title,content,complete_texts
0,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"NvidiaSearchInput.mount({""apiUrl"": ""https://ap...",CUDA Toolkit Documentation 12.5\nNvidiaSearchI...
1,https://docs.nvidia.com/cuda/cuda-toolkit-rele...,CUDA 12.5 Update 1 Release Notes,1. CUDA 12.5 Update 1 Release Notes \r\n 1.1. ...,CUDA 12.5 Update 1 Release Notes\n1. CUDA 12.5...
2,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Linux,1. Introduction \n 1.1. System Requirements \n...,CUDA Installation Guide for Linux\n1. Introduc...
3,https://docs.nvidia.com/cuda/cuda-features-arc...,CUDA Features Archive,1. CUDA 11.6 Features \n \n 1.1. Compiler \n 1...,CUDA Features Archive\n1. CUDA 11.6 Features \...
4,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Microsoft Windows,1. Introduction \n 1.1. System Requirements \n...,CUDA Installation Guide for Microsoft Windows\...


In [7]:
texts = df.complete_texts.to_list()

len(texts)

4856

In [19]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings()
text_splitter = SemanticChunker(hf_embeddings)
docs = text_splitter.create_documents(texts)

In [ ]:
from  langchain.schema import Document
import json
from typing import Iterable

def save_docs_to_jsonl(array:Iterable[Document], file_path:str)->None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')

save_docs_to_jsonl(docs,'chunked_documents.jsonl')

In [ ]:
import re
def clean_string(s):
    cleaned_lines = [' '.join(line.split()) for line in s.splitlines()]
    cleaned_string = '\n'.join(cleaned_lines)
    cleaned_string = re.sub(r'\n+', '\n', cleaned_string)
    return cleaned_string

In [ ]:
def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data)
            array.append(obj)
    return array

docs = load_docs_from_jsonl('chunked_documents.jsonl')

In [ ]:
with open("embedded_chunks.jsonl", "a") as f:
    for i, document in enumerate(docs):
        vector = {
            "id": i,
            "vector":hf_embeddings.embed_documents([clean_string(document.page_content)])[0],
            "chunk": clean_string(document.page_content)
        }
        f.write(json.dumps(vector) + "\n")

In [20]:
client.list_collections()

[]

In [1]:
import pandas as pd

embedded_chunks = pd.read_json("embedded_chunks.jsonl", lines=True)

embedded_chunks.head()

,id,vector,chunk
0,0,"[-0.030554667115211, -0.047709718346595, -0.02...",CUDA Toolkit Documentation 12.5\nNvidiaSearchI...
1,1,"[-0.03192675486207, -0.058551348745822004, -0....",CUB The user guide for CUB. CUDA C++ Standard ...
2,2,"[-0.011653744615614001, -0.008399281650781, -0...",cuSPARSE The cuSPARSE library user guide. NPP ...
3,3,"[-0.022603930905461003, -0.041946601122617, -0...",nvFatbin The user guide for the nvFatbin libra...
4,4,"[-0.015180315822362001, -0.038234677165746, -0...",Nsight Visual Studio Edition The documentation...


In [2]:
len(max(embedded_chunks.vector.to_list(), key=len))

768

In [4]:
len(max(embedded_chunks.chunk.to_list(), key=len))

532055

In [5]:
embedded_chunks["chunk_len"] = embedded_chunks["chunk"].apply(len)

embedded_chunks["chunk_len"].describe(percentiles=[0.25, 0.5, 0.75, 0.85, 0.9, 0.95])

count     90168.000000
mean       2096.602242
std        4139.977184
min           1.000000
25%         233.000000
50%         874.000000
75%        2594.000000
85%        3949.000000
90%        5301.000000
95%        7784.650000
max      532055.000000
Name: chunk_len, dtype: float64

In [6]:
embedded_chunks = embedded_chunks[embedded_chunks["chunk_len"]<5000]
embedded_chunks["chunk_len"].describe(percentiles=[0.25, 0.5, 0.75, 0.85, 0.9, 0.95])

count    80188.000000
mean      1174.857161
std       1233.951333
min          1.000000
25%        204.000000
50%        682.000000
75%       1826.000000
85%       2668.950000
90%       3180.000000
95%       3853.000000
max       4999.000000
Name: chunk_len, dtype: float64

In [7]:
embedded_chunks = embedded_chunks[["id", "vector", "chunk"]]

In [8]:
embedded_chunks.head()

,id,vector,chunk
1,1,"[-0.03192675486207, -0.058551348745822004, -0....",CUB The user guide for CUB. CUDA C++ Standard ...
2,2,"[-0.011653744615614001, -0.008399281650781, -0...",cuSPARSE The cuSPARSE library user guide. NPP ...
3,3,"[-0.022603930905461003, -0.041946601122617, -0...",nvFatbin The user guide for the nvFatbin libra...
4,4,"[-0.015180315822362001, -0.038234677165746, -0...",Nsight Visual Studio Edition The documentation...
5,5,"[-0.024751845747232003, -0.029467459768056002,...",Compiler SDK \nlibNVVM API The libNVVM API. l...


In [5]:
from pymilvus import MilvusClient, DataType
import os

CLUSTER_ENDPOINT = "https://in03-6f08bdb85fec6ea.api.gcp-us-west1.zillizcloud.com"
TOKEN = os.getenv('ZILLIZ_TOKEN')

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN 
)

In [6]:
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=768)
schema.add_field(field_name="chunk", datatype=DataType.VARCHAR, max_length=8000)


client.create_collection(
    collection_name="cuda_data", 
    schema=schema, 
)

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="HNSW",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name="cuda_data",
    index_params=index_params
)

In [7]:
res = client.list_indexes(
    collection_name="cuda_data"
)

print(res)

['vector_index']


In [8]:
res = client.describe_index(
    collection_name="cuda_data",
    index_name="vector_index"
)

res

{'nlist': '128',
 'index_type': 'HNSW',
 'metric_type': 'COSINE',
 'field_name': 'vector',
 'index_name': 'vector_index'}

In [37]:
data = embedded_chunks.to_json(orient="records")

In [38]:
import json

data = json.loads(data)

In [39]:
data_part1 = data[:len(data)//6]
data_part2 = data[len(data)//6:2*len(data)//6]
data_part3 = data[2*len(data)//6:3*len(data)//6]
data_part4 = data[3*len(data)//6:4*len(data)//6]
data_part5 = data[4*len(data)//6:5*len(data)//6]
data_part6 = data[5*len(data)//6:]

In [40]:
len(data_part1), len(data_part2), len(data_part3), len(data_part4), len(data_part5), len(data_part6) 

(13364, 13365, 13365, 13364, 13365, 13365)

In [41]:
res = client.insert(
    collection_name="cuda_data",
    data=data_part1
)

In [42]:
res = client.insert(
    collection_name="cuda_data",
    data=data_part2
)
res = client.insert(
    collection_name="cuda_data",
    data=data_part3
)
res = client.insert(
    collection_name="cuda_data",
    data=data_part4
)
res = client.insert(
    collection_name="cuda_data",
    data=data_part5
)
res = client.insert(
    collection_name="cuda_data",
    data=data_part6
)

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings()
question = "What is the CUDA version suppoorted by NVIDIA RTX"
embedded_query = hf_embeddings.embed_query(question)

In [10]:
embedded_query

[-0.007597993593662977,
 -0.0635538399219513,
 -0.024716200307011604,
 0.04655836150050163,
 -0.053499311208724976,
 -0.010638721287250519,
 0.0255771242082119,
 0.03909706696867943,
 -0.02049882523715496,
 -0.0034186614211648703,
 0.024846207350492477,
 -0.02604740858078003,
 0.005014731083065271,
 0.03797614201903343,
 -0.0328238420188427,
 -0.013636168092489243,
 0.010999892838299274,
 0.0032550024334341288,
 -0.004016126040369272,
 -0.025991166010499,
 0.0023851371370255947,
 -0.03240218013525009,
 0.03409785032272339,
 -0.005184631794691086,
 0.013919355347752571,
 -0.03328476846218109,
 0.03320198878645897,
 -0.028282644227147102,
 0.021814409643411636,
 0.05087016150355339,
 0.06302899122238159,
 0.03022865578532219,
 0.07306563854217529,
 0.050070036202669144,
 1.5749211570437183e-06,
 -0.035719916224479675,
 -0.04585469141602516,
 0.0062783462926745415,
 0.03299633786082268,
 -0.043623052537441254,
 0.035127393901348114,
 -0.011887396685779095,
 -0.014432089403271675,
 -0.0273

In [11]:
res = client.search(
    collection_name="cuda_data", 
    data=[embedded_query],
    limit=5, 
    search_params={"metric_type": "COSINE", "params": {}}
)

In [12]:
ids = [i["id"] for i in res[0]]
ids

[79977, 80478, 79595, 28101, 65301]

In [13]:
res = client.get(
    collection_name="cuda_data",
    ids=ids
)

In [14]:
res_chunks = [i["chunk"] for i in res]

res_chunks

["NVIDIA CUDA Toolkit Version Support\nThe releases in this release family of NVIDIA vGPU software support NVIDIA CUDA Toolkit 10.2. For more information about NVIDIA CUDA Toolkit, see CUDA Toolkit 10.2 Documentation . Note:\nIf you are using NVIDIA vGPU software with CUDA on Linux, avoid conflicting installation methods by installing CUDA from a distribution-independent runfile package. Do not install CUDA from distribution-specific RPM or Deb package. To ensure that the NVIDIA vGPU software graphics driver is not overwritten when CUDA is installed, deselect the CUDA driver when selecting the CUDA components to install. For more information, see NVIDIA CUDA Installation Guide for Linux . 2.5. Multiple vGPU Support\nTo support applications and workloads that are compute or graphics intensive, multiple vGPUs can be added to a single VM. The assignment of more than one vGPU to a VM is supported only on a subset of vGPUs and Red Hat Enterprise Linux with KVM releases. Supported vGPUs\nOnl

In [20]:
import requests
import os
API_URL = "https://api-inference.huggingface.co/models/BAAI/bge-m3"
headers = {"Authorization": "Bearer "+str(os.getenv('HUGGINGFACE_TOKEN'))}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
	"source_sentence": question,
	"sentences":list(res_chunks)
},
})

In [21]:
output

[0.6791619658470154,
 0.6657265424728394,
 0.6852164268493652,
 0.6842777729034424,
 0.6716938018798828]

In [25]:
print("Question: "+question)
print("Context: "+res_chunks[output.index(max(output))])

Question: What is the CUDA version suppoorted by NVIDIA RTX
Context: NVIDIA CUDA Toolkit version supported: 11.6
See NVIDIA CUDA Toolkit and OpenCL Support on NVIDIA vGPU Software in Virtual GPU Software User Guide for details about supported features and limitations. For a list of validated server platforms, refer to NVIDIA GRID Certified Servers . Citrix Hypervisor Support
Linux with KVM Support
Microsoft Windows Server Support
Nutanix AHV Support
Red Hat Enterprise Linux with KVM Support
Ubuntu Support
VMware vSphere ESXi Support
Citrix Hypervisor Support
Driver Package
Hypervisor or Bare-Metal OS
Software Product Deployment
Hardware Supported
Guest OS Support 1 , 2
Supported Virtualization Products 3 , 4
NVIDIA vGPU for XenServer 8.2 5
Citrix Hypervisor 8.2
NVIDIA vGPU
GPU pass through
A2
A10
A16
A30 7 , A30X 7
A40 6 , A100 7 , A100X 7
M6, M10, M60
P4, P6, P40, P100, P100 12GB
T4
V100
RTX A5000 6
RTX A5500 6
RTX A6000 6
RTX 6000, RTX 6000 passive, RTX 8000, RTX 8000 passive 6
Windo

In [28]:
context = res_chunks[output.index(max(output))]

In [26]:
from huggingface_hub import InferenceClient
import json


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
    token=str(os.getenv('HUGGINGFACE_TOKEN'))
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 300},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [33]:
prompt = """
Your task is to answer a question given a context.
Question asked: """+ question+"""
Context provided to you from NVIDIA Documentation site: """+ context[:500]+"""
Your answer is:
"""

In [34]:
res = call_llm(llm_client,prompt)

In [35]:
res.split("\n")[-1]

'The NVIDIA RTX supports CUDA Toolkit version 11.6.'